In [154]:
import buffett_code_com as bc

import requests
import json
import pandas as pd
from datetime import *

In [155]:
# Generate company data
def generate_df_c(ticker:str):
    res_c = bc.fetch(bc.API_ENDPOINT_COMPANY, ticker,  None, None)
    json_data_c = json.loads(res_c.text)
    # json に保管
    with open('json/company.json', 'w') as f:
        json.dump(json_data_c, f)
    df_c = pd.DataFrame.from_dict(json_data_c[ticker])
    return df_c

In [74]:
# API からの取得 JSON への保存
df_c = generate_df_c("7974")
df_c

,tosyo_33category,url,company_name,company_name_en,fiscal_month,established_date,listing_date,priority_market,accounting_standard,latest_fiscal_year,latest_fiscal_quarter,oldest_fiscal_year,oldest_fiscal_quarter,fixed_tier_range
0,その他製品,https://www.nintendo.co.jp/,任天堂,"""Nintendo Co., Ltd""",3.0,1947/11,1962/01/13,t1,日本,2020.0,2.0,2004.0,1.0,"{'oldest_fiscal_year': 2015.0, 'oldest_fiscal_..."


In [126]:
# 全期間の有価証券情報を Dynamo に追加する

In [156]:
# JSON からの取得
json_c_all = json.load(open('json/company.json', 'r'),  parse_float=decimal.Decimal)
json_c_target = json_c_all["7974"]
df_c = pd.DataFrame.from_dict(json_c_target)
df_c["fixed_tier_range"][0]

{'oldest_fiscal_year': Decimal('2015.0'),
 'oldest_fiscal_quarter': Decimal('4.0'),
 'latest_fiscal_year': Decimal('2020.0'),
 'latest_fiscal_quarter': Decimal('2.0')}

In [157]:
import decimal
# Generate quarter data
def generate_df_q(ticker:str, start_q:str, end_q:str):
    res_q = bc.fetch(bc.API_ENDPOINT_QUARTER, ticker, start_q, end_q)
    json_data_q = json.loads(res_q.text,  parse_float=decimal.Decimal)
    df_q = pd.DataFrame.from_dict(json_data_q[ticker])
    # 日付データを datetime型に変換
    df_q['to_datetime'] = pd.to_datetime(df_q['edinet_updated_date'])
    df_sorted = sorted(df_q['to_datetime'] )
    df_concat = pd.DataFrame()
    for sort_num in range(len(df_sorted)):
        datetime_data = df_q[df_q['to_datetime'] == df_sorted[sort_num]]
        df_concat = pd.concat([df_concat, datetime_data], axis=0)
    return df_q

In [158]:
#  oldest_fiscal_year / oldest_fiscal_quarter= 2004.0 1.0
# latest_fiscal_year / latest_fiscal_quarter = 2020.0 2.0
df_q = generate_df_q("7974", "2015Q4", "2018Q2")
df_q
# for column in df_q.columns:
#    print(column)

,company_name,ceo_name,headquarters_address,end_date,updated_date,edinet_updated_date,tdnet_updated_date,edinet_title,tdnet_title,accounting_standard,...,employee_num,net_sales_per_employee,operating_income_per_employee,segment_member,increase_in_properties,r_and_d_expenses,desc_business,fiscal_year,fiscal_quarter,to_datetime
0,任天堂株式会社,取締役社長 君島 達己,東京都台東区浅草橋５丁目21番５号任天堂株式会社 東京支店,2016-03-31T00:00:00.000Z,2016-06-30T00:00:00.000Z,2016-06-30T00:00:00.000Z,2016-04-27T16:00:00.000Z,【E02367】任天堂株式会社 有価証券報告書－第76期(平成27年4月1日－平成28年3月...,【79740】任天堂株式会社 平成28年3月期 決算短信〔日本基準〕（連結）,Japan GAAP,...,5064.0,99616706.16113745,6493088.467614534,None,4670000000.0,None,"<h3 class=""smt_head2"">３ 【事業の内容】</h3><p class=""...",2015.0,4.0,2016-06-30 00:00:00+00:00
1,任天堂株式会社,代表取締役社長 君島 達己,東京都台東区浅草橋5丁目21番5号任天堂株式会社 東京支店,2016-06-30T00:00:00.000Z,2016-08-09T00:00:00.000Z,2016-08-09T00:00:00.000Z,2016-07-27T16:00:00.000Z,【E02367】任天堂株式会社 四半期報告書－第77期第1四半期(平成28年4月1日－平成2...,【79740】任天堂株式会社 平成29年3月期 第1四半期決算短信〔日本基準〕（連結）,Japan GAAP,...,None,None,None,None,0.0,None,None,2016.0,1.0,2016-08-09 00:00:00+00:00
2,任天堂株式会社,代表取締役社長 君島 達己,東京都台東区浅草橋5丁目21番5号任天堂株式会社 東京支店,2016-09-30T00:00:00.000Z,2016-11-11T00:00:00.000Z,2016-11-11T00:00:00.000Z,2016-10-26T16:00:00.000Z,【E02367】任天堂株式会社 四半期報告書－第77期第2四半期(平成28年7月1日－平成2...,【79740】任天堂株式会社 平成29年3月期 第2四半期決算短信〔日本基準〕（連結）,Japan GAAP,...,None,None,None,None,6958000000.0,None,None,2016.0,2.0,2016-11-11 00:00:00+00:00
3,任天堂株式会社,代表取締役社長 君島 達己,東京都台東区浅草橋5丁目21番5号任天堂株式会社 東京支店,2016-12-31T00:00:00.000Z,2017-02-13T00:00:00.000Z,2017-02-13T00:00:00.000Z,2017-01-31T16:00:00.000Z,【E02367】任天堂株式会社 四半期報告書－第77期第3四半期(平成28年10月1日－平成...,【79740】任天堂株式会社 平成29年3月期 第3四半期決算短信〔日本基準〕（連結）,Japan GAAP,...,None,None,None,None,0.0,None,None,2016.0,3.0,2017-02-13 00:00:00+00:00
4,任天堂株式会社,代表取締役社長 君島 達己,東京都台東区浅草橋5丁目21番5号任天堂株式会社 東京支店,2017-03-31T00:00:00.000Z,2017-06-30T09:12:00.000Z,2017-06-30T09:12:00.000Z,2017-04-27T16:00:00.000Z,【E02367】任天堂株式会社 有価証券報告書－第77期(平成28年4月1日－平成29年3月...,【79740】任天堂株式会社 平成29年3月期 決算短信〔日本基準〕（連結）,Japan GAAP,...,5166.0,94675764.61478901,5683701.122725513,None,10458000000.0,None,"\n<p style=""page-break-before:always; line-hei...",2016.0,4.0,2017-06-30 09:12:00+00:00
5,任天堂株式会社,代表取締役社長 君島 達己,東京都台東区浅草橋5丁目21番5号任天堂株式会社 東京支店,2017-06-30T00:00:00.000Z,2017-08-10T10:51:00.000Z,2017-08-10T10:51:00.000Z,2017-07-26T16:00:00.000Z,【E02367】任天堂株式会社 四半期報告書－第78期第1四半期(平成29年4月1日－平成2...,【79740】任天堂株式会社 平成30年3月期第1四半期決算短信〔日本基準〕（連結）,Japan GAAP,...,None,None,None,None,0.0,None,"\n<p class=""smt_text2"">当第1四半期連結累計期間において、当社及び当社...",2017.0,1.0,2017-08-10 10:51:00+00:00
6,任天堂株式会社,代表取締役社長 君島 達己,東京都台東区浅草橋5丁目21番5号任天堂株式会社 東京支店,2017-09-30T00:00:00.000Z,2017-11-10T13:38:00.000Z,2017-11-10T13:38:00.000Z,2017-10-30T16:00:00.000Z,【E02367】任天堂株式会社 四半期報告書－第78期第2四半期(平成29年7月1日－平成2...,【79740】任天堂株式会社 平成30年3月期第2四半期決算短信〔日本基準〕（連結）,Japan GAAP,...,None,None,None,None,4192000000.0,None,"\n<p class=""smt_text2"">当第2四半期連結累計期間において、当社及び当社...",2017.0,2.0,2017-11-10 13:38:00+00:00
7,任天堂株式会社,代表取締役社長 君島 達己,東京都台東区浅草橋5丁目21番5号任天堂株式会社 東京支店,2017-12-31T00:00:00.000Z,2018-02-13T11:29:00.000Z,2018-02-13T11:29:00.000Z,2018-01-31T16:00:00.000Z,【E02367】任天堂株式会社 四半期報告書－第78期第3四半期(平成29年10月1日－平成...,【79740】任天堂株式会社 平成30年3月期第3四半期決算短信〔日本基準〕（連結）,Japan GAAP,...,None,None,None,None,0.0,None,"\n<p class=""smt_text2"">当第3四半期連結累計期間において、当社及び当社...",2017.0,3.0,2018-02-13 11:29:00+00:00
8,任天堂株式会社,代表取締役社長 古川 俊太郎,東京都台東区浅草橋5丁目21番5号任天堂株式会社 東京支店,2018-03-31T00:00:00.000Z,2018-06-29T11:58:00.000Z,2018-06-29T11:58:00.000Z,2018-04-26T16:00:00.000Z,【E02367】任天堂株式会社 有価証券報告書－第78期(平成29年4月1日－平成30年3月...,【79740】任天堂株式会社 平成30年3月期決算短信〔日本基準〕（連結）,Japan GAAP,...,5501.0,191907289.58371204,32277222.32321396,None,9609000000.0,None,"\n<p style=""page-break-before:always; line-hei...",2017.0,4.0,2018-06-29 11:58:00+00:00
9,任天堂株式会社,代表取締役社長 古川 俊太郎,東京都台東区浅草橋5丁目21番5号任天堂株式会社 東京支店,2018-06-30T00:00:00.000Z,2018-08-10T09:00:00.000Z,2018-08-10T09:00:00.000Z,2018-07-31T16:00:00.000Z,【E02367】任天堂株式会社 四半期報告書－第79期第1四半期(平成30年4月1日－平成3...,【79740】任天堂株式会社 平成31年3月期第1四半期決算短信〔日本基準〕（連結）,Japan GAAP,...

In [159]:
# Dynamoへ追加する

In [162]:
import json
import boto3

def store_df_q(df_q, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.Table('Quarter')

    df_q["ticker"] = 7974
    # df_q["to_datetime"] = df_q["to_datetime"].dt.strftime("%Y%m%d")
    df_q = df_q.drop(["end_date","updated_date","edinet_updated_date","tdnet_updated_date", "to_datetime"], axis=1)
    for index, row in df_q.iterrows():
        row["fiscal_year_quarter"] = f"{int(row['fiscal_year'])}Q{int(row['fiscal_quarter'])}"
        #print(f"Adding movie : {ticker}{fiscal_year}{fiscal_quarter}")
        table.put_item(Item=row.to_dict())

In [163]:
# if __name__ == '__main__':
store_df_q(df_q)

In [164]:
# 項目を読み込む

In [181]:
from pprint import pprint
import boto3
from botocore.exceptions import ClientError

def get_df_q(ticker:int, fiscal_year_quarter:str, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.Table('Quarter')

    try:
        response = table.get_item(Key={'ticker': ticker, 'fiscal_year_quarter': fiscal_year_quarter})
    except ClientError as e:
        print(e.response['Error']['Message'])
    else:
        return response['Item']

df_q = get_df_q(7974, "2015Q4")
if df_q:
    print("Get Quarter succeeded:")
    pprint(df_q)

Get Quarter succeeded:
{'accounting_standard': 'Japan GAAP',
 'accounts_payable': None,
 'accounts_receivable': None,
 'accounts_receivable_turnover': Decimal('28.023714513964464'),
 'accrual': Decimal('-37610000000'),
 'additional_capital_stock': Decimal('13256000000'),
 'adjusted_debt_ratio': Decimal('9.635456518905194'),
 'advances_received': None,
 'amortization': None,
 'assets': Decimal('1296902000000'),
 'bonds_issuance': None,
 'bonds_payable': None,
 'bonds_repayment': None,
 'bps': Decimal('9662.72822498134'),
 'buildings': Decimal('39977000000'),
 'capital_stock': Decimal('10065000000'),
 'cash_and_deposits': Decimal('570448000000'),
 'cash_assets_ratio': Decimal('43.985436062246805'),
 'cash_monthly_sales_ratio': Decimal('13.569737084678835'),
 'cash_translation_difference': Decimal('-3898000000'),
 'ccc': Decimal('39.06407162910622'),
 'ceo_name': '取締役社長\xa0\xa0\xa0 君島\xa0 達己',
 'commercial_papers_liabilities': None,
 'company_name': '任天堂株式会社',
 'construction_in_progress':

In [ ]:
# クエリ - tickerのみ

In [182]:
import boto3
from boto3.dynamodb.conditions import Key # Boto 3 SDK によって Attr が自動的に作成される

def get_df_q(ticker, dynamodb=None):
    if not dynamodb:
        dynamodb = boto3.resource('dynamodb', endpoint_url="http://localhost:8000")

    table = dynamodb.Table('Quarter')
    response = table.query(
        KeyConditionExpression=Key('ticker').eq(ticker)
    )
    return response['Items']

ticker = 7974
print(f"Data with ticker : {ticker}")
df_q_list = get_df_q(ticker)
for df_q in df_q_list:
    pprint(df_q)

Data with ticker : 7974
{'accounting_standard': 'Japan GAAP',
 'accounts_payable': None,
 'accounts_receivable': None,
 'accounts_receivable_turnover': Decimal('28.023714513964464'),
 'accrual': Decimal('-37610000000'),
 'additional_capital_stock': Decimal('13256000000'),
 'adjusted_debt_ratio': Decimal('9.635456518905194'),
 'advances_received': None,
 'amortization': None,
 'assets': Decimal('1296902000000'),
 'bonds_issuance': None,
 'bonds_payable': None,
 'bonds_repayment': None,
 'bps': Decimal('9662.72822498134'),
 'buildings': Decimal('39977000000'),
 'capital_stock': Decimal('10065000000'),
 'cash_and_deposits': Decimal('570448000000'),
 'cash_assets_ratio': Decimal('43.985436062246805'),
 'cash_monthly_sales_ratio': Decimal('13.569737084678835'),
 'cash_translation_difference': Decimal('-3898000000'),
 'ccc': Decimal('39.06407162910622'),
 'ceo_name': '取締役社長\xa0\xa0\xa0 君島\xa0 達己',
 'commercial_papers_liabilities': None,
 'company_name': '任天堂株式会社',
 'construction_in_progress'

                  '<p class="smt_text6" '
                  'style="text-align:justify;padding-right:0pt;margin-top:0pt;margin-bottom:0pt;font-family:&apos;ＭＳ '
                  '明朝&apos;;letter-spacing:0pt;line-height:13pt;">\u3000</p>\n'
                  '<p class="smt_head6" '
                  'style="padding-left:13.5pt;padding-right:0pt;margin-top:0pt;margin-bottom:0pt;text-justify:inter-ideograph;text-indent:0pt;font-family:&apos;ＭＳ '
                  '明朝&apos;;text-align:justify;letter-spacing:0pt;line-height:13pt;">[開発]</p>\n'
                  '<p class="smt_text6" '
                  'style="text-align:justify;padding-left:18pt;padding-right:0pt;margin-top:0pt;margin-bottom:0pt;text-indent:0pt;font-family:&apos;ＭＳ '
                  '明朝&apos;;letter-spacing:0pt;line-height:13pt;">任天堂株式会社、Nintendo '
                  'Technology Development Inc.、Nintendo Software Technology '
                  'Corporation、<br/>Retro Studios, Inc.、Nintendo European '
                  'Re

 'adjusted_debt_ratio': Decimal('19.685463518252785'),
 'advances_received': None,
 'amortization': None,
 'assets': Decimal('1633474000000'),
 'bonds_issuance': None,
 'bonds_payable': None,
 'bonds_repayment': None,
 'bps': Decimal('10980.44167359295'),
 'buildings': Decimal('36094000000'),
 'capital_stock': Decimal('10065000000'),
 'cash_and_deposits': Decimal('744555000000'),
 'cash_assets_ratio': Decimal('45.58107444624157'),
 'cash_monthly_sales_ratio': Decimal('8.463400910501457'),
 'cash_translation_difference': Decimal('1222000000'),
 'ccc': Decimal('26.258328126483818'),
 'ceo_name': '代表取締役社長\xa0\xa0\xa0 古川\xa0 俊太郎',
 'commercial_papers_liabilities': None,
 'company_name': '任天堂株式会社',
 'construction_in_progress': Decimal('653000000'),
 'convertible_bond_type_bonds_with_subscription_rights': None,
 'convertible_bonds': None,
 'corporate_tax_payable': Decimal('43390000000'),
 'cost_of_sales': Decimal('652141000000'),
 'current_allowance_doubtful_accounts': Decimal('-87000000'),


In [188]:
pd.DataFrame.from_dict(df_q_list)

,treasury_stock,edinet_title,good_will,ex_ordinary_income,ccc,gross_margin,loss_of_sales_non_current_assets,inventory_turnover,work_in_process,net_debt,...,net_income_growth_rate_forecast,notes_accounts_receivable,long_term_loans_payable,net_income,ceo_name,investments_and_other_assets,sga_ratio,r_and_d_ratio,impairment_loss,net_short_term_debt
0,-250563000000,【E02367】任天堂株式会社 有価証券報告書－第76期(平成27年4月1日－平成28年3月...,None,45000000000,39.06407162910622,43.80237046023562,None,52.056410364946,30000000,-570448000000,...,112.05695243865495,38731000000,None,16518000000,取締役社長 君島 達己,178037000000,37.28410039269792,0,None,None
1,-250567000000,【E02367】任天堂株式会社 四半期報告書－第77期第1四半期(平成28年4月1日－平成2...,None,45000000000,23.392724764918576,49.26979618841679,None,45.439322253503654,None,-582532000000,...,-242.65916687046544,25351000000,None,-24530000000,代表取締役社長 君島 達己,204183000000,57.556197453565495,0,None,None
2,-250580000000,【E02367】任天堂株式会社 四半期報告書－第77期第2四半期(平成28年7月1日－平成2...,None,10000000000,7.117752858499998,45.01067157851651,None,61.698466903517264,1441000000,-584545000000,...,30.551711532938185,32850000000,None,38305000000,代表取締役社長 君島 達己,235888000000,49.3575124989036,0,None,None
3,-250594000000,【E02367】任天堂株式会社 四半期報告書－第77期第3四半期(平成28年10月1日－平成...,None,30000000000,0.8509426631737256,45.0194618813902,None,50.08047981198979,None,-660476000000,...,-12.595052879993007,85494000000,None,102979000000,代表取締役社長 君島 達己,230924000000,36.5613378717605,0,None,None
4,-250601000000,【E02367】任天堂株式会社 有価証券報告書－第77期(平成28年4月1日－平成29年3月...,None,60000000000,-2.675884339934697,40.666537175804294,None,49.21387884781717,117000000,-662763000000,...,-56.12923352896446,106054000000,None,102582000000,代表取締役社長 君島 達己,228851000000,34.66300003066889,0,None,None
5,-252435000000,【E02367】任天堂株式会社 四半期報告書－第78期第1四半期(平成29年4月1日－平成2...,None,60000000000,-11.415686004342149,42.012994177933265,None,35.39510627302991,None,-701997000000,...,111.6650987770461,57763000000,None,21795000000,代表取締役社長 君島 達己,237692000000,31.492383282814846,0,None,None
6,-250628000000,【E02367】任天堂株式会社 四半期報告書－第78期第2四半期(平成29年7月1日－平成2...,None,125000000000,-1.238064853237674,38.422525872832125,None,75.88343625074945,46000000,-683407000000,...,65.0389297710813,97542000000,None,52672000000,代表取締役社長 君島 達己,248709000000,27.73893771003714,0,None,None
7,-250662000000,【E02367】任天堂株式会社 四半期報告書－第78期第3四半期(平成29年10月1日－平成...,None,175000000000,-0.9373607725180193,38.07472940868973,None,52.99807096931492,None,-782723000000,...,-11.219620463877478,225241000000,None,137122000000,代表取締役社長 君島 達己,271252000000,19.817925536631925,0,None,None
8,-250679000000,【E02367】任天堂株式会社 有価証券報告書－第78期(平成29年4月1日－平成30年3月...,None,230000000000,26.258328126483818,38.22552624748741,None,79.36138043766609,63000000,-744555000000,...,18.20330969267139,69829000000,None,140945000000,代表取締役社長 古川 俊太郎,269597000000,21.40635153388994,0,None,None
9,-250684000000,【E02367】任天堂株式会社 四半期報告書－第79期第1四半期(平成30年4月1日－平成3...,None,230000000000,51.09174333064213,49.303329626480014,None,111.08239558459756,None,-632061000000,...,439.21568627450984,77832000000,None,30580000000,代表取締役社長 古川 俊太郎,284217000000,31.144704056328308,0,None,None
